In [2]:
import numpy as np
import cv2
import time
from matplotlib import pyplot as plt

In [22]:
# u, v are N-by-2 matrices, representing N corresponding points for v = T(u)
# this function should return a 3-by-3 homography matrix
def solve_homography(u, v, solve='solution1'):
    N = u.shape[0]
    if v.shape[0] is not N:
        print('u and v should have the same size')
        return None
    if N < 4:
        print('At least 4 points should be given')
    
    b = np.zeros((2*N, 1))
    H = np.zeros((3, 3))
    # TODO: compute H from A and b
    if solve == 'solution1':
        A = np.zeros((2*N, 8))
        for i in range(N):
            b[2*i : 2*(i+1)] = v[i].reshape(-1, 1)

        for i in range(N):
            A[2*i, 0] = u[i, 0]
            A[2*i, 1] = u[i, 1]
            A[2*i, 2] = 1.
            A[2*i, 6] = -u[i, 0] * v[i, 0]
            A[2*i, 7] = -u[i, 1] * v[i, 0]
            b[2*i] = v[i, 0]

            A[2*i+1, 3] = u[i, 0]
            A[2*i+1, 4] = u[i, 1]
            A[2*i+1, 5] = 1.
            A[2*i+1, 6] = -u[i, 0] * v[i, 1]
            A[2*i+1, 7] = -u[i, 1] * v[i, 1]

        h = np.linalg.solve(A, b)
        H[:2, :] = h[:6].reshape(2, 3)
        H[2, 0] = h[6]
        H[2, 1] = h[7]
        H[2, 2] = 1.
    
    elif solve == 'solution2':
        A = np.zeros((2*N, 9))
        for i in range(N):
            A[2*i, 0] = u[i, 0]
            A[2*i, 1] = u[i, 1]
            A[2*i, 2] = 1.
            A[2*i, 6] = -u[i, 0] * v[i, 0]
            A[2*i, 7] = -u[i, 1] * v[i, 0]
            A[2*i, 8] = -v[i, 0]

            A[2*i+1, 3] = u[i, 0]
            A[2*i+1, 4] = u[i, 1]
            A[2*i+1, 5] = 1.
            A[2*i+1, 6] = -u[i, 0] * v[i, 1]
            A[2*i+1, 7] = -u[i, 1] * v[i, 1]
            A[2*i+1, 8] = -v[i, 1]  

        # solve by SVD decomposition
        u, s, vh = np.linalg.svd(A)
        v = vh.transpose()
        H = v[:, -1].reshape(3, 3)
    return H


# corners are 4-by-2 arrays, representing the four image corner (x, y) pairs
def transform(img, canvas, corners):
    h, w, ch = img.shape
    # TODO:
    img_corners = np.array([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    H = solve_homography(img_corners, corners)
    
    for y in range(h):
        for x in range(w):
            # convert the pixel values of img to the new location
            pixel = img[y, x, :]
            u = np.array([[x, y, 1]])
            v = np.dot(H, u.T)
            t_x = int(v[0] / v[2])
            t_y = int(v[1] / v[2])
            canvas[t_y, t_x, :] = pixel
            

def backWarp(img, output, corners, interpolate='bilinear'):
    h, w, ch = output.shape
    output_corners = np.array([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    H = solve_homography(output_corners, corners)

    for y in range(h):
        for x in range(w):
            u = np.array([[x, y, 1]])
            v = np.dot(H, u.T)
            t_x = (v[0] / v[2])[0]
            t_y = (v[1] / v[2])[0]
            # because t_x, t_y are decimal numbers, we should use interpolation to assign them
            # to a discrete number.
            if interpolate == 'bilinear':
                # assisgn the pixel values in img to the output
                output[y, x, :] = bilinear(img, t_x, t_y)
            elif interpolate == 'nearest':
                t_x, t_y = round(t_x), round(t_y)
                output[y, x, :] = img[t_y, t_x, :]
            
            
def bilinear(img, t_x, t_y):
    int_x, int_y = int(t_x), int(t_y)
    x_left = round(t_x - int_x, 3)
    x_right = 1 - x_left
    y_low = round(t_y - int_y, 3)
    y_high = 1 - y_low
    
    img_low_left = x_left * y_low * img[int_y + 1, int_x + 1]
    img_low_right = x_right * y_low * img[int_y + 1, int_x]
    img_high_left = x_left * y_high * img[int_y, int_x + 1]
    img_high_right = x_right * y_high * img[int_y, int_x]
    value = img_low_left + img_low_right + img_high_left + img_high_right
    return value

    
def main():
    ### Part 1 ###
    ts = time.time()
    canvas = cv2.imread('./input/Akihabara.jpg')
    img1 = cv2.imread('./input/lu.jpeg')
    img2 = cv2.imread('./input/kuo.jpg')
    img3 = cv2.imread('./input/haung.jpg')
    img4 = cv2.imread('./input/tsai.jpg')
    img5 = cv2.imread('./input/han.jpg')

    canvas_corners1 = np.array([[779,312],[1014,176],[739,747],[978,639]])  # (col, row)
    canvas_corners2 = np.array([[1194,496],[1537,458],[1168,961],[1523,932]])
    canvas_corners3 = np.array([[2693,250],[2886,390],[2754,1344],[2955,1403]])
    canvas_corners4 = np.array([[3563,475],[3882,803],[3614,921],[3921,1158]])
    canvas_corners5 = np.array([[2006,887],[2622,900],[2008,1349],[2640,1357]])
    
    # TODO: 
    transform(img1, canvas, canvas_corners1)
    transform(img2, canvas, canvas_corners2)
    transform(img3, canvas, canvas_corners3)
    transform(img4, canvas, canvas_corners4)
    transform(img5, canvas, canvas_corners5)

    cv2.imwrite('part1_test2.png', canvas)
    te = time.time()
    print('Elapse time: {}...'.format(te-ts))

    
    ### Part 2 ###
    ts = time.time()
    img = cv2.imread('./input/QR_code.jpg')
    # TODO: 
    output2 = np.zeros((200, 200, 3))
    QR_corners = np.array([[1980, 1237], [2040, 1212], [2025, 1395], [2083, 1363]])
    backWarp(img, output2, QR_corners)

    cv2.imwrite('part2.png', output2)
    te = time.time()
    print('Elapse time: {}...'.format(te-ts))

    
    ### Part 3 ###
    ts = time.time()
    img_front = cv2.imread('./input/crosswalk_front.jpg')
    # TODO: 
    output3 = np.zeros((300, 500, 3))
    crosswalk_corners = np.array([[150, 148], [572, 146], [5, 297], [723, 291]])
    backWarp(img_front, output3, crosswalk_corners)

    cv2.imwrite('part3.png', output3)
    te = time.time()
    print('Elapse time: {}...'.format(te-ts))
    

In [23]:
if __name__ == '__main__':
    main()

Elapse time: 5.111350774765015...
